Transformer From Scratch 
========================

Reading along Dan Jurafsky and James H. Martin's [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book, I decided to follow through Chapter 8 of their book to implement a Transformer using Pytorch. It is my goal to have a working transformer which I can use to train on the guitar dataset. I know some linear algebra, the book essentially gives the entire algorithm in terms of linear algebra, and pytorch provides a nice but still very informative abstractions for doing linear algebra. I had no reason not to pursue this project on top of whatever I proposed to do initially. 

## 

Attention Layer
---------------

At the heart of Transformer is the **attention layer**. It is a mechanism that allows words(tokens) to gain contextual meaning from their surrounding words(tokens). It can have multiple **"heads"**, where each "head" can be thought of as a specialist who asks particular set of questions given some data. For instance, one head could focus solely on grammar while another could instead focus on sentiments (even though that might not be exactly what occurs under the hood).

Each head's job, then, is to ask the right kinds of *questions* to choose which of previous words it has seen matters the most to the current word. To do this, each head consists of three main components: **Query**, **Key**, and **Value** weight matrices. 

<!-- 
    essentially, what it is at the end of the day is weighted sum, but it's obviously lot more complicated than that
    don't forget to write out the equations that I have referenced
    maybe throw in some pictures
    say something about how masking and softmax is used to determine what key's to focus on
    also explain how results from different heads are consolidated at the end
--!>

In [2]:
from myTransformer import *
batch_size = 10
N = 10
model_dim = 24
num_heads = 4
key_dim = 3

M = 8
X = torch.rand((batch_size, N, model_dim)) # batch_size is 10, 3 words represented as dim (1, 4) tensors
Y = torch.rand((batch_size, M, model_dim)) # 3 words represented as dim (1, 4) tensors
mask = torch.tensor([[0 if i>= j else -torch.inf for j in range(N)] for i in range(N)])

multihead_attention = AttentionLayer(model_dim=model_dim, key_dim=key_dim, num_heads=num_heads)
multihead_attention(X, H_enc=Y, mask=mask).shape
#multihead_attention.to("cuda")
#multihead_attention(X.to("cuda"), Y.to("cuda"), mask=mask.to("cuda"))

torch.Size([10, 10, 24])

In [2]:
encoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads)
decoder_block = TransformerBlock(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, cross_attention=True)
encoder_block(X,mask=mask)
decoder_block(X, Y)

tensor([[[ 5.1096e-01,  5.5467e-01,  0.0000e+00,  ...,  5.4671e-01,
           5.5573e-01,  7.6994e-01],
         [ 9.6602e-01,  5.3381e-01,  1.5186e+00,  ...,  2.8803e-01,
           1.0655e+00,  8.7264e-01],
         [ 1.5144e+00, -0.0000e+00,  1.2530e+00,  ...,  0.0000e+00,
           0.0000e+00,  1.8795e-01],
         ...,
         [ 0.0000e+00,  1.8030e-01,  9.9155e-01,  ...,  9.7477e-01,
           7.5042e-01,  8.8840e-01],
         [ 1.1271e+00, -4.2854e-02,  3.1018e-01,  ...,  9.2582e-01,
           0.0000e+00,  1.0774e+00],
         [ 1.0861e+00, -1.4413e-02,  1.2386e+00,  ...,  4.3418e-01,
           1.4451e+00,  1.0363e+00]],

        [[ 0.0000e+00, -2.4558e-01,  0.0000e+00,  ...,  4.5616e-01,
           5.0151e-01,  8.5969e-02],
         [ 0.0000e+00, -7.0843e-02,  9.3438e-01,  ...,  5.6688e-01,
           0.0000e+00,  4.1849e-01],
         [ 1.4262e+00,  2.2593e-01,  1.5120e+00,  ...,  6.1659e-01,
           8.7490e-01,  0.0000e+00],
         ...,
         [ 7.5110e-01,  1

In [3]:
stack = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=8, num_heads=num_heads, num_stack=9)
stack.state_dict()
stack.train()
stack(X)

tensor([[[-3.4192e-01,  0.0000e+00, -0.0000e+00,  ..., -7.5203e-01,
          -8.1255e-01, -1.1268e+00],
         [-9.2207e-02,  4.9095e+00,  1.5488e+00,  ..., -2.2938e-01,
          -2.7140e+00, -1.3915e+00],
         [ 4.1725e-01, -4.5254e-01,  1.6255e+00,  ..., -1.1403e+00,
          -7.1456e-01, -2.1576e+00],
         ...,
         [ 0.0000e+00, -0.0000e+00, -4.9746e-01,  ..., -1.0125e+00,
          -2.4384e+00, -1.0519e+00],
         [-1.0844e+00,  2.2505e+00,  1.3393e+00,  ..., -4.5376e-01,
          -6.5878e-01, -1.7609e+00],
         [-0.0000e+00,  2.0725e+00,  8.2439e-01,  ..., -3.1636e+00,
          -5.2293e-01, -1.3149e-01]],

        [[ 1.2173e+00,  8.2994e-01,  4.9181e+00,  ..., -3.4747e+00,
          -1.0330e+00, -1.5496e+00],
         [-3.5608e-01, -5.7887e-01, -6.8485e-01,  ..., -2.7861e+00,
           6.5649e-01, -2.9923e+00],
         [ 7.4284e-02,  1.9172e+00, -5.9658e-01,  ..., -1.1080e+00,
           9.0089e-01, -5.5628e-01],
         ...,
         [-9.7783e-01, -1

In [4]:
# from https://pytorch-tutorials-preview.netlify.app/beginner/transformer_tutorial.html
# i don't completely understand positional encoding yet, but I have built the intuition that 
# it is analogous to how binary numbers encode numbers; smaller bits flips more frequently 
# than larger bits; this is modeled by the sinusodial waves 
# it also takes advantage of linearity of trigonometric addition formulas, which supposedly 
# helps the model to figure out relative positioning...
# https://medium.com/thedeephub/positional-encoding-explained-a-deep-dive-into-transformer-pe-65cfe8cfe10b 
class PositionalEncoding(nn.Module):

    def __init__(self, model_dim: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_dim, 2) * (-math.log(10000.0) / model_dim))
        pe = torch.zeros(max_len, 1, model_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, X):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        X = X + self.pe[:X.size(0)]
        return self.dropout(X)

In [5]:
# what I need to do to finish up the encoder_decoder architecture 
# the only difference for the decoder architecture is the cross attention layer `
# which is much like the self-attension layer except that it is using both the final 
# H of the encoder and that of decoder to do query-key matching, thus decoder needs to 
# take in memory from encoder
pos = PositionalEncoding(model_dim=model_dim)


In [3]:
import dac
from ICMTSMTGuitarData import *
from jupyter_audio_utils import *
import torch

model_path = dac.utils.download(model_type="44khz")
model = dac.DAC.load(model_path, weights_only=True).eval()

mono_data2 = ICMTSMTGuitarDataMono(DEFAULT_PEDAL_PROBS)
test_2 = mono_data2[random.randint(0, len(mono_data2)-1)] 

waveform, sr = test_2[0] 
play_audio(*test_2[0])

x = model.preprocess(waveform, sr)

with torch.no_grad():
    z, codes, latents, _, _ = model.encode(x.unsqueeze(dim=0))

audio_tokens = z.transpose(-2,-1) #(batch_size, seq_len, model_dim)
N = audio_tokens.shape[-2] 
model_dim = 32

# further compress information or is this just baseless; the hope is that this will distill features that actually matter
linear = torch.nn.Linear(in_features=audio_tokens.shape[-1], out_features=model_dim)

key_dim = 256
hidden_dim = 512
num_heads=4
print(audio_tokens.shape)

compressed = linear(audio_tokens)
print(compressed.shape)
encoder = TransformerStack(N=N, model_dim=model_dim, key_dim=key_dim, hidden_dim=256, num_heads=num_heads, num_stack=1)
latent = testing(compressed)

/home/yuc3/miniconda3/envs/dac_compatible/lib/python3.8/site-packages/audiotools/ml/layers/base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load

torch.Size([1, 173, 1024])
torch.Size([1, 173, 32])


In [7]:
from myTransformer import *

# blueprint
class EncoderDecoder(nn.Module): 
    def __init__(
        self, 
        N, 
        model_dim,
        key_dim,
        encoder_mask,
        encoder_hidden_dim, 
        encoder_num_stack,
        encoder_num_heads,
        decoder_hidden_dim, 
        decoder_num_stack,
        decoder_num_heads,
        decoder_vocab
    ):
        pass
        self.positional_encoder = PositionalEncoding()
        self.encoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=encoder_hidden_dim, 
            num_heads=encoder_num_heads, 
            num_stack=encoder_num_stack
        )
        self.decoder_stack = TransformerStack(
            N=N, 
            model_dim=model_dim, 
            key_dim=key_dim, 
            hidden_dim=decoder_hidden_dim, 
            num_heads=decoder_num_heads, 
            num_stack=decoder_num_stack,
            cross_attention=True
        )
        self.language_head = None
        self.decoder_vocab = decoder_vocab
        self.mask = encoder_mask # do the self register thingy
    def forward(self, X): # (batch_size, N, model_dim)
        X = self.positional_encoder(X)
        H1 = self.encoder_stack(X, mask=self.mask)
        H2 = self.decoder_stack(X, H1)
        Y = self.language_head(H2,...) # not implemented yet

        return Y